In [1]:
import pandas as pd

def parse_conllu_file(filepath):
    data = []
    with open(filepath, "r", encoding="utf-8") as file:
        sent_id = text = twittiro = None
        for line in file:
            line = line.strip()
            if line.startswith("# sent_id ="):
                sent_id = line.split("=", 1)[1].strip()
            elif line.startswith("# text ="):
                text = line.split("=", 1)[1].strip()
            elif line.startswith("# twittiro ="):
                twittiro = line.split("=", 1)[1].strip()

                if "IMPLICIT" in twittiro or "EXPLICIT" in twittiro:
                    #replace
                    twittiro = twittiro.replace("IMPLICIT", "").replace("EXPLICIT", "")

                # remove first \t
                if twittiro.startswith("\t"):
                    twittiro = twittiro[1:]

                # Quando abbiamo tutti e tre i dati, li salviamo
                if sent_id and text and twittiro:
                    data.append({
                        "id": sent_id,
                        "input": text,
                        "output": twittiro
                    })
                    # Reset per la prossima frase
                    sent_id = text = twittiro = None
    return pd.DataFrame(data)

# Esempio di utilizzo
datasets = {
    'dev': 'it_twittiro-ud-dev.conllu',
    'test': 'it_twittiro-ud-test.conllu',
    'train': 'it_twittiro-ud-train.conllu',   
}

for split in datasets:
    df = parse_conllu_file(datasets[split])
    df.to_csv(f"{split}.csv", index=False, encoding="utf-8")